# Imports

In [1]:
import pandas as pd
import numpy as np
import torch

# Supporting Functions

In [2]:
# Definition of file names to import and export
input_csv, relevant_values_csv, output_csv = 'data_train.csv', 'sampleSubmission.csv', 'result.csv'

In [3]:
def convert_csv_to_matrix(input_csv, format):
    df = pd.read_csv(input_csv)
    
    df['row'] = df['Id'].apply(lambda x: int(x.split('_')[0][1:]))
    df['col'] = df['Id'].apply(lambda x: int(x.split('_')[1][1:]))
    
    max_row = df['row'].max()
    max_col = df['col'].max()

    if(format == "zero"):
        matrix = np.zeros((max_row, max_col))
        for index, row in df.iterrows():
            matrix[row['row']-1, row['col']-1] = row['Prediction']

    else:
        # Initialize and populate dictionary to store rows
        row_dict = {i: {} for i in range(1, max_row + 1)}
        for index, row in df.iterrows():
            row_dict[row['row']][row['col']] = row['Prediction']


        matrix = np.full((max_row, max_col), np.nan)
        for r in range(1, max_row + 1):
            for c in range(1, max_col + 1):
                if c in row_dict[r]:
                    matrix[r-1, c-1] = row_dict[r][c]
    
    return matrix

In [4]:
def save_matrix_to_csv(matrix, relevant_values_csv, output_csv):
    # Import the relevant values csv and convert to dataframe
    relevant_values_df = pd.DataFrame(convert_csv_to_matrix(relevant_values_csv, 'zero'))
    
    matrix_df = pd.DataFrame(matrix)
    
    # Create a filtered version of the matrix. The criteria used is: relevant_values_df == 3
    filtered_matrix_df = matrix_df.where(relevant_values_df == 3, other=np.nan)
    
    # Reshape the matrix into one column and reset the index; also removes NaN values
    stacked_df = filtered_matrix_df.stack().reset_index()
    
    # Rename the columns
    stacked_df.columns = ['row', 'col', 'val']
    
    # Create the desired rN_cN format for the final output
    stacked_df['row'] = (stacked_df['row'] + 1).astype(str)
    stacked_df['col'] = (stacked_df['col'] + 1).astype(str)
    stacked_df['r_c'] = 'r' + stacked_df['row'] + '_c' + stacked_df['col']
    
    result_df = stacked_df[['r_c', 'val']]
    
    result_df.to_csv(output_csv, index=False, header=['Id', 'Prediction'])

In [5]:
# impute missing entries og the matrix with the row mean
def mean_matrix(matrix):
    for r in range(matrix.shape[0]):
        row_mean = np.nanmean(matrix[r])
        matrix[r] = np.where(np.isnan(matrix[r]), row_mean, matrix[r])
    return matrix


# Choose Parameters

In [75]:
mode = "singular_value_thresholding"  # All options: "singular_values_fixed", "singular_values_shrink","singular_value_thresholding"
num_iterations = 20
eta = 1
center_data = False             # Whether to center the data before SVD; only applicable for modes singular_values_fixed and singular_values_shrink 
shrinkage_tau = 42             # Used in the modes singular_values_shrink and singular_value_thresholding
num_keep_singular_values = 8    # Used in the mode singular_values_fixed
activate_validation = True    # Use test/validation sets for loss

# Iterative SVD Algorithms

Adapted from the lecture script and exercise materials

In [76]:
# Returns the singular values matrix S with all values smaller than tau being set to 0
def shrink(S):
    S[:] -= shrinkage_tau
    return torch.clamp(S, min=0)

In [77]:
# Returns an SVD approximation of a given matrix, controlled by the parameters shrinkage_tau, num_keep_singular_values and mode
def svd_approximation(matrix):
    
    # SVD Decomposition
    U, S, Vh = torch.linalg.svd(matrix, full_matrices=False)
    
    if mode == "singular_values_fixed":
    # Only keep the largest num_keep_singular_values singular values, all others are set to zero
        S[-1*(min(list(matrix.shape)) - num_keep_singular_values):] = 0
    
    elif mode == "singular_values_shrink" or mode == "singular_value_thresholding":
    # Nuclear Norm
        S = shrink(S)

    else:
        raise ValueError("Unknown mode")
    # Return the matrix calculated by the low rank approximation
    return U @ torch.diag(S) @ Vh

In [78]:
# Import matrix from CSV, impute missing values by the row means, and convert into a torch tensor
matrix = convert_csv_to_matrix(input_csv, "svd")
torch_matrix = torch.from_numpy(matrix)

# Create mask of the known values
known_values_mask = ~torch.isnan(torch_matrix)

testing_values_mask = known_values_mask.clone()

validation_size_relative = 0.1               # size of validation set relative to total size
validation_size = round(validation_size_relative * testing_values_mask.shape[1])
train_size = round((1 - validation_size_relative) * testing_values_mask.shape[1])
total_size = train_size + validation_size
if activate_validation:
    # Define validation and training sets
    testing_values_mask[:,:train_size] = False # validation data mask: The last validation_size percent of the users
    known_values_mask[:,train_size:] = False # training data mask: The other users coming before

In [79]:
# Impute the missing values of the matrix by the row mean
svd_matrix = torch.from_numpy(mean_matrix(matrix))

# Iterate through SVD approximating and re-filling the known original values
for iteration in range(num_iterations):
    if mode == "singular_values_fixed" or mode == "singular_values_shrink":
        # Re-fill the known original values with variable eta
        svd_matrix[known_values_mask] = svd_matrix[known_values_mask] + eta * (torch_matrix[known_values_mask] - svd_matrix[known_values_mask])
        
        if center_data:
            # Subtract mean from matrix
            matrix_mean = svd_matrix.mean(dim=1, keepdim=True)
            svd_matrix_centered = svd_matrix - matrix_mean
            
            # Calculate SVD approximation, shift by previously subtracted mean and clamp values to [1,5]
            svd_matrix = torch.clamp(svd_approximation(svd_matrix_centered) + matrix_mean, min=1.0, max=5.0)
        else:
            # Calculate SVD approximation and clamp values to [1,5]
            svd_matrix = torch.clamp(svd_approximation(svd_matrix), min=1.0, max=5.0)
        
        # Metric on how close the approximated values match the known values
        if activate_validation:
            print(f'ITERATION: {iteration:03}, Validation Loss: {round(torch.dist(svd_matrix[testing_values_mask], torch_matrix[testing_values_mask]).item()/validation_size,5)}, Training Loss: {round(torch.dist(svd_matrix[known_values_mask], torch_matrix[known_values_mask]).item()/train_size,5)}')
        else:
            print(f'ITERATION: {iteration:03}, Loss: {round(torch.dist(svd_matrix[testing_values_mask], torch_matrix[testing_values_mask]).item()/total_size,5)}')
       
    
    elif mode == "singular_value_thresholding":
        # Follow procedure as described in formula (2.41), mean imputed matrix used as A0
        matrix_shrink = svd_approximation(svd_matrix)
        svd_matrix[known_values_mask] = svd_matrix[known_values_mask] + eta * (torch_matrix[known_values_mask] - matrix_shrink[known_values_mask])
        
        # Metric on how close the approximated values match the known values
        if activate_validation:
            print(f'ITERATION: {iteration:03}, Validation Loss: {round(torch.dist(svd_approximation(svd_matrix)[testing_values_mask], torch_matrix[testing_values_mask]).item()/validation_size,5)}, Training Loss: {round(torch.dist(svd_approximation(svd_matrix)[known_values_mask], torch_matrix[known_values_mask]).item()/train_size,5)}')
        else:    
            print(f'ITERATION: {iteration:03}, Loss: {round(torch.dist(svd_approximation(svd_matrix)[testing_values_mask]/total_size, torch_matrix[testing_values_mask]).item()/total_size,5)}')
        
    else:
        raise ValueError("Unknown mode")
    
# Output matrix
if mode == "singular_values_fixed" or mode == "singular_values_shrink":
    matrix_out = svd_matrix
elif mode == "singular_value_thresholding":
    matrix_out = torch.clamp(svd_approximation(svd_matrix), min=1.0, max=5.0)
else:
    raise ValueError("Unknown mode")
    
# Save the matrix to the output CSV
save_matrix_to_csv(matrix_out, relevant_values_csv, output_csv)

ITERATION: 000, Validation Loss: 2.67522, Training Loss: 1.30366
ITERATION: 001, Validation Loss: 2.62749, Training Loss: 1.36008
ITERATION: 002, Validation Loss: 2.60589, Training Loss: 1.37094
ITERATION: 003, Validation Loss: 2.5955, Training Loss: 1.37358
ITERATION: 004, Validation Loss: 2.58896, Training Loss: 1.37535
ITERATION: 005, Validation Loss: 2.58422, Training Loss: 1.37669
ITERATION: 006, Validation Loss: 2.58057, Training Loss: 1.37775
ITERATION: 007, Validation Loss: 2.57766, Training Loss: 1.37862
ITERATION: 008, Validation Loss: 2.57529, Training Loss: 1.37935
ITERATION: 009, Validation Loss: 2.5733, Training Loss: 1.37999
ITERATION: 010, Validation Loss: 2.57161, Training Loss: 1.38054
ITERATION: 011, Validation Loss: 2.57016, Training Loss: 1.38103
ITERATION: 012, Validation Loss: 2.5689, Training Loss: 1.38147
ITERATION: 013, Validation Loss: 2.56779, Training Loss: 1.38186
ITERATION: 014, Validation Loss: 2.56681, Training Loss: 1.38222
ITERATION: 015, Validation L